In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.pipeline import Pipeline

In [2]:
# Load your preprocessed dataset
df = pd.read_excel("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/data-sets/4. Scored Classified Preprocessed DATA.xlsx")  # replace with your actual path

In [3]:
# Features and labels
X = df.drop(columns=['Emergency_Level', 'Score'])
y_cls = df['Emergency_Level']
y_reg = df['Score']

In [4]:
# Split dataset
X_train, X_test, y_cls_train, y_cls_test, y_reg_train, y_reg_test = train_test_split(
    X, y_cls, y_reg, test_size=0.2, random_state=42
)

In [5]:
# ---- CLASSIFICATION ---- #
# Define base models
clf1 = SVC(probability=True, random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = KNeighborsClassifier()

# Voting Classifier
voting_clf = VotingClassifier(
    estimators=[('svm', clf1), ('dt', clf2), ('knn', clf3)],
    voting='soft'
)

In [6]:
# Fit classification model
voting_clf.fit(X_train, y_cls_train)
y_cls_pred = voting_clf.predict(X_test)

print("Classification Report:")
print(classification_report(y_cls_test, y_cls_pred))

Classification Report:
                    precision    recall  f1-score   support

     Emergent (L2)       0.91      0.95      0.93       490
  Less Urgent (L4)       0.97      0.83      0.89       180
   Non-Urgent (L5)       0.98      0.97      0.98        64
Resuscitation (L1)       0.92      0.94      0.93       543
       Urgent (L3)       1.00      0.98      0.99       116

          accuracy                           0.93      1393
         macro avg       0.96      0.93      0.94      1393
      weighted avg       0.93      0.93      0.93      1393



In [7]:
# ---- REGRESSION WITHIN EACH CLASS ---- #
# Append predicted emergency levels
X_test_copy = X_test.copy()
X_test_copy['Predicted_Class'] = y_cls_pred
X_test_copy['True_Score'] = y_reg_test.values

In [8]:
# Dictionary to store models for each class
regressors = {}

# Loop over each class and train a regressor
for level in sorted(df['Emergency_Level'].unique()):
    print(f"\nTraining regressor for Emergency_Level = {level}")
    X_class = X_train[y_cls_train == level]
    y_class = y_reg_train[y_cls_train == level]

    # GradientBoostingRegressor is a good choice for regression here
    reg = GradientBoostingRegressor(random_state=42)
    reg.fit(X_class, y_class)
    regressors[level] = reg

    # Optional: Evaluate on training class subset
    preds = reg.predict(X_class)
    print(f"RMSE for level {level}: {np.sqrt(mean_squared_error(y_class, preds))}")


Training regressor for Emergency_Level = Emergent (L2)
RMSE for level Emergent (L2): 3.3132748734393505

Training regressor for Emergency_Level = Less Urgent (L4)
RMSE for level Less Urgent (L4): 2.478525607514781

Training regressor for Emergency_Level = Non-Urgent (L5)
RMSE for level Non-Urgent (L5): 1.4055504093766171

Training regressor for Emergency_Level = Resuscitation (L1)
RMSE for level Resuscitation (L1): 3.4527072074174145

Training regressor for Emergency_Level = Urgent (L3)
RMSE for level Urgent (L3): 2.3739718130185388


In [9]:
import joblib
import os

# Create a directory to store models
os.makedirs("/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models", exist_ok=True)

# Save the classification model
joblib.dump(voting_clf, "/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/classifier_voting_model.pkl")

# Save each regressor by class
for level, model in regressors.items():
    joblib.dump(model, f"/home/abdeldjalil-hani/Desktop/emergency-sorting-system/saved-models/regressor_level_{level}.pkl")

print("All models saved successfully.")

All models saved successfully.
